# Football Player Project

In [1]:
# Import relevant libraries.
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [2]:
# Load the players_combined.csv dataset.
df = pd.read_csv('players_combined.csv')

# Set display options to show all columns.
pd.set_option('display.max_columns', None)

# View the df DataFrame shape and head.
print("The DataFrame shape:", df.shape)
df.head(20)

The DataFrame shape: (18136, 56)


,id,name,age,nationality,club,position,height,weight,preferred_foot,work_rate,body_type,joined,loaned_from,contract_valid_until,value,release_clause,wage,overall,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,fk_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,international_reputation,weak_foot,skill_moves
0,185149,J. Koprivec,29,Slovenia,No Club,GK,6'1,179lbs,Right,Medium/ Medium,Normal,NaN,NaN,NaN,NaN,NaN,NaN,68,68,15,14,10,22,12,18,16,11,31,23,51,49,48,60,40,28,70,26,61,17,27,19,9,32,21,64,12,13,14,69,65,60,68,72,1,1,1
1,189024,M. Banguera,32,Ecuador,No Club,GK,6'2,198lbs,Right,Medium/ Medium,Stocky,NaN,NaN,NaN,NaN,NaN,NaN,73,73,12,14,15,14,16,18,19,20,12,12,40,40,28,69,31,12,35,30,65,14,30,18,16,21,13,39,15,16,14,68,70,65,77,77,1,3,1
2,201572,M. Korhut,29,Hungary,No Club,LB,5'11,168lbs,Left,High/ Medium,Normal,NaN,NaN,NaN,NaN,NaN,NaN,71,71,72,28,59,56,24,66,41,36,53,69,76,75,68,72,69,24,64,71,66,26,72,66,55,52,35,65,69,72,72,14,15,7,11,12,1,3,3
3,205211,M. Borjan,30,Canada,No Club,GK,6'5,185lbs,Right,Medium/ Medium,Normal,NaN,NaN,NaN,NaN,NaN,NaN,73,73,15,12,15,38,19,28,16,16,48,23,47,40,38,70,42,25,55,55,77,12,53,22,14,57,23,58,19,23,21,75,71,71,73,72,1,2,1
4,205868,R. Kirilov,26,Bulgaria,No Club,LW,5'7,132lbs,Right,Medium/ Low,Lean,NaN,NaN,NaN,NaN,NaN,NaN,65,66,65,53,59,65,51,71,56,39,59,71,79,79,89,53,89,53,55,68,33,50,29,18,55,51,58,53,36,23,14,6,9,13,5,14,1,3,3
5,207439,L. Paredes,24,Argentina,No Club,CM,5'11,165lbs,Right,Medium/ Medium,Normal,NaN,NaN,NaN,NaN,NaN,NaN,80,85,76,55,60,84,73,78,79,78,82,82,75,69,77,74,77,82,61,79,69,80,79,72,74,82,57,74,73,75,72,9,14,6,9,10,2,4,4
6,208485,L. Ralte,25,India,No Club,LW,5'8,157lbs,Left,High/ Low,Lean,NaN,NaN,NaN,NaN,NaN,NaN,61,62,60,54,44,55,49,60,55,54,48,62,80,77,71,58,72,66,39,55,58,62,36,43,57,56,56,54,36,29,27,9,13,8,9,8,1,3,2
7,216477,T. Nedelev,25,Bulgaria,No Club,RM,5'9,154lbs,Left,Medium/ Medium,Normal,NaN,NaN,NaN,NaN,NaN,NaN,68,73,65,66,49,67,53,73,69,68,60,68,78,75,78,58,76,72,54,60,50,67,28,23,62,66,64,59,43,25,22,10,9,8,7,10,1,3,3
8,217225,J. Lalpekhlua,27,India,No Club,RS,5'9,165lbs,Right,Medium/ Medium,Stocky,NaN,NaN,NaN,NaN,NaN,NaN,63,64,61,63,57,57,50,61,47,58,46,64,77,74,79,57,80,70,52,60,63,64,47,34,55,46,65,47,28,31,29,13,11,10,10,11,1,3,2
9,236457,D. Giannoulis,22,Greece,No Club,LB,5'10,143lbs,Left,High/ Medium,Lean,NaN,NaN,NaN,NaN,NaN,NaN,65,73,62,42,59,55,37,62,44,40,60,58,73,79,64,63,73,65,59,73,47,38,64,65,58,54,47,58,62,62,63,14,5,13,9,12,1,2,2


In [3]:
# View the metadata of the df DataFrame.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18136 entries, 0 to 18135
Data columns (total 56 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        18136 non-null  int64  
 1   name                      18136 non-null  object 
 2   age                       18136 non-null  int64  
 3   nationality               18136 non-null  object 
 4   club                      18136 non-null  object 
 5   position                  18136 non-null  object 
 6   height                    18136 non-null  object 
 7   weight                    18136 non-null  object 
 8   preferred_foot            18136 non-null  object 
 9   work_rate                 18136 non-null  object 
 10  body_type                 18136 non-null  object 
 11  joined                    16643 non-null  object 
 12  loaned_from               1264 non-null   object 
 13  contract_valid_until      16643 non-null  float64
 14  value 

As you can see above, importing the dataset into Python has removed the 'N/A' values from the 'loaned_from' column. This needs to be corrected once the dataset is imported into Power BI.

In [4]:
# Handle missing values and categorical features.
X = df[['age', 'overall', 'potential', 'position']]  # Features.
y = df['value']  # Target.

# Split the dataset into rows with known and missing 'value'.
train_data = df[df['value'].notnull()]  # Rows with known values.
X_train = train_data[['age', 'overall', 'potential', 'position']]
y_train = train_data['value']

# Data with missing values in 'value' column for prediction.
missing_data = df[df['value'].isnull()]
X_missing = missing_data[['age', 'overall', 'potential', 'position']]

# Define the preprocessing pipeline.
preprocessor = ColumnTransformer(transformers=[
        ('num', SimpleImputer(strategy='mean'), ['age', 'overall', 'potential']),  # Impute missing numeric values.
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), ['position'])])  # OneHotEncoder for 'position'.

In [5]:
# Create the Random Forest model pipeline.
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(random_state=42))])

# Train the Random Forest model on known values.
model_pipeline.fit(X_train, y_train)

# Predict on the rows with missing values.
predictions = model_pipeline.predict(X_missing)

# Evaluate the Random Forest model on training data.
y_train_pred = model_pipeline.predict(X_train)

# Calculate metrics for Random Forest model.
mae = mean_absolute_error(y_train, y_train_pred)
mse = mean_squared_error(y_train, y_train_pred)
rmse = mean_squared_error(y_train, y_train_pred, squared=False)
r2 = r2_score(y_train, y_train_pred)

# Output evaluation metrics for Random Forest model.
print(f"Random Forest - MAE: {mae}, MSE: {mse}, RMSE: {rmse}, R2: {r2}")

Random Forest - MAE: 55837.91158196265, MSE: 101145115947.5019, RMSE: 318033.1994422939, R2: 0.9968124441860214


The Random Forest model demonstrates strong predictive performance with an R² value of 0.9968, indicating that 99.68% of the variance in the target variable is explained by the model. The MAE of 55,837.91 and RMSE of 318,033.20 reflect the average and root mean square prediction errors, respectively, with the RMSE being more sensitive to large deviations. While the MSE of 101,145,115,947.50 is high due to squaring errors, the overall metrics suggest the model fits the data very well, though careful attention is needed to assess real-world relevance and overfitting risks.

In [6]:
# Create the Gradient Boosting model pipeline.
gb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Reuse the existing preprocessor.
    ('model', GradientBoostingRegressor(random_state=42))])

# Train the Gradient Boosting model on known values.
gb_pipeline.fit(X_train, y_train)

# Predict on rows with missing values using Gradient Boosting.
gb_predictions = gb_pipeline.predict(X_missing)

# Evaluate the Gradient Boosting model on training data.
y_train_pred_gb = gb_pipeline.predict(X_train)

# Calculate metrics for Gradient Boosting model.
mae_gb = mean_absolute_error(y_train, y_train_pred_gb)
mse_gb = mean_squared_error(y_train, y_train_pred_gb)
rmse_gb = mean_squared_error(y_train, y_train_pred_gb, squared=False)
r2_gb = r2_score(y_train, y_train_pred_gb)

# Output evaluation metrics for Gradient Boosting model.
print(f"Gradient Boosting - MAE: {mae_gb}, MSE: {mse_gb}, RMSE: {rmse_gb}, R2: {r2_gb}")

Gradient Boosting - MAE: 209196.40946721667, MSE: 231934770755.14227, RMSE: 481596.06596726086, R2: 0.9926906502596916


The Gradient Boosting model shows strong predictive performance with an R² value of 0.9927, meaning it explains 99.27% of the variance in the target variable. The MAE of 209,196.41 indicates the average error, while the RMSE of 481,596.07 highlights larger prediction deviations. The MSE of 231,934,770,755.14 is higher due to squared error calculations, reflecting the sensitivity to outliers. While the model performs well, its errors are notably larger compared to the Random Forest model, suggesting potential room for improvement in capturing complex patterns.

In [7]:
# Detect and remove outliers using IQR.
Q1 = X_train[['age', 'overall', 'potential']].quantile(0.25)
Q3 = X_train[['age', 'overall', 'potential']].quantile(0.75)
IQR = Q3 - Q1

# Filter the data to remove outliers.
outlier_condition = ((X_train[['age', 'overall', 'potential']] < (Q1 - 1.5 * IQR)) |
                     (X_train[['age', 'overall', 'potential']] > (Q3 + 1.5 * IQR))).any(axis=1)

# Remove outliers from X_train and y_train.
X_train_no_outliers = X_train[~outlier_condition]
y_train_no_outliers = y_train[~outlier_condition]

# Create the Random Forest model pipeline without outliers.
model_pipeline_no_outliers = Pipeline(steps=[
    ('preprocessor', preprocessor), # Reuse the existing preprocessor.
    ('model', RandomForestRegressor(random_state=42))])

# Train the Random Forest model on data without outliers.
model_pipeline_no_outliers.fit(X_train_no_outliers, y_train_no_outliers)

# Predict on the rows with missing values.
predictions_no_outliers = model_pipeline_no_outliers.predict(X_missing)

# Evaluate the Random Forest model on training data without outliers.
y_train_pred_no_outliers = model_pipeline_no_outliers.predict(X_train_no_outliers)

# Calculate metrics for the Random Forest model without outliers.
mae_no_outliers = mean_absolute_error(y_train_no_outliers, y_train_pred_no_outliers)
mse_no_outliers = mean_squared_error(y_train_no_outliers, y_train_pred_no_outliers)
rmse_no_outliers = mean_squared_error(y_train_no_outliers, y_train_pred_no_outliers, squared=False)
r2_no_outliers = r2_score(y_train_no_outliers, y_train_pred_no_outliers)

# Output evaluation metrics for the Random Forest model without outliers.
print(f"Random Forest without Outliers - MAE: {mae_no_outliers}, MSE: {mse_no_outliers}, "
      f"RMSE: {rmse_no_outliers}, R2: {r2_no_outliers}")

Random Forest without Outliers - MAE: 38462.46172308438, MSE: 19519516139.496857, RMSE: 139712.26195111457, R2: 0.9985660397294006


The Random Forest model without outliers achieves exceptional performance, with an R² value of 0.9986, explaining 99.86% of the variance in the target variable. The MAE of 38,462.46 and RMSE of 139,712.26 indicate significantly lower prediction errors compared to the Random Forest model that included outliers, highlighting improved accuracy. The MSE of 19,519,516,139.50 is also markedly reduced, reflecting the model's ability to better handle the adjusted data distribution. These results demonstrate that removing outliers substantially enhances the model's predictive precision and reliability.

In [8]:
# Predict the missing values using the Random Forest model without outliers.
# Round to 0 decimal place to keep consistent with the other values in the 'value' column.
df.loc[df['value'].isnull(), 'value'] = model_pipeline_no_outliers.predict(X_missing).round(0)

# Check if the 'value' column has any missing values.
missing_values = df['value'].isnull().sum()

# Output the result.
print(f"Number of missing values in 'value' column: {missing_values}")

Number of missing values in 'value' column: 0


In [9]:
# View the metadata of the df DataFrame.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18136 entries, 0 to 18135
Data columns (total 56 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        18136 non-null  int64  
 1   name                      18136 non-null  object 
 2   age                       18136 non-null  int64  
 3   nationality               18136 non-null  object 
 4   club                      18136 non-null  object 
 5   position                  18136 non-null  object 
 6   height                    18136 non-null  object 
 7   weight                    18136 non-null  object 
 8   preferred_foot            18136 non-null  object 
 9   work_rate                 18136 non-null  object 
 10  body_type                 18136 non-null  object 
 11  joined                    16643 non-null  object 
 12  loaned_from               1264 non-null   object 
 13  contract_valid_until      16643 non-null  float64
 14  value 

In [10]:
# Issues existed when importing dataset into Power BI.
# Fill missing values in the specified columns with pd.NA,
# and convert the data types to 'Int64' for handling nullable integers.
df['contract_valid_until'] = df['contract_valid_until'].fillna(pd.NA).astype('Int64')
df['value'] = df['value'].fillna(pd.NA).astype('Int64')
df['release_clause'] = df['release_clause'].fillna(pd.NA).astype('Int64')
df['wage'] = df['wage'].fillna(pd.NA).astype('Int64')

# View the df DataFrame head.
df.head(20)

,id,name,age,nationality,club,position,height,weight,preferred_foot,work_rate,body_type,joined,loaned_from,contract_valid_until,value,release_clause,wage,overall,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,fk_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,international_reputation,weak_foot,skill_moves
0,185149,J. Koprivec,29,Slovenia,No Club,GK,6'1,179lbs,Right,Medium/ Medium,Normal,NaN,NaN,<NA>,600250,<NA>,<NA>,68,68,15,14,10,22,12,18,16,11,31,23,51,49,48,60,40,28,70,26,61,17,27,19,9,32,21,64,12,13,14,69,65,60,68,72,1,1,1
1,189024,M. Banguera,32,Ecuador,No Club,GK,6'2,198lbs,Right,Medium/ Medium,Stocky,NaN,NaN,<NA>,2348000,<NA>,<NA>,73,73,12,14,15,14,16,18,19,20,12,12,40,40,28,69,31,12,35,30,65,14,30,18,16,21,13,39,15,16,14,68,70,65,77,77,1,3,1
2,201572,M. Korhut,29,Hungary,No Club,LB,5'11,168lbs,Left,High/ Medium,Normal,NaN,NaN,<NA>,1815000,<NA>,<NA>,71,71,72,28,59,56,24,66,41,36,53,69,76,75,68,72,69,24,64,71,66,26,72,66,55,52,35,65,69,72,72,14,15,7,11,12,1,3,3
3,205211,M. Borjan,30,Canada,No Club,GK,6'5,185lbs,Right,Medium/ Medium,Normal,NaN,NaN,<NA>,2600000,<NA>,<NA>,73,73,15,12,15,38,19,28,16,16,48,23,47,40,38,70,42,25,55,55,77,12,53,22,14,57,23,58,19,23,21,75,71,71,73,72,1,2,1
4,205868,R. Kirilov,26,Bulgaria,No Club,LW,5'7,132lbs,Right,Medium/ Low,Lean,NaN,NaN,<NA>,621667,<NA>,<NA>,65,66,65,53,59,65,51,71,56,39,59,71,79,79,89,53,89,53,55,68,33,50,29,18,55,51,58,53,36,23,14,6,9,13,5,14,1,3,3
5,207439,L. Paredes,24,Argentina,No Club,CM,5'11,165lbs,Right,Medium/ Medium,Normal,NaN,NaN,<NA>,19510000,<NA>,<NA>,80,85,76,55,60,84,73,78,79,78,82,82,75,69,77,74,77,82,61,79,69,80,79,72,74,82,57,74,73,75,72,9,14,6,9,10,2,4,4
6,208485,L. Ralte,25,India,No Club,LW,5'8,157lbs,Left,High/ Low,Lean,NaN,NaN,<NA>,298250,<NA>,<NA>,61,62,60,54,44,55,49,60,55,54,48,62,80,77,71,58,72,66,39,55,58,62,36,43,57,56,56,54,36,29,27,9,13,8,9,8,1,3,2
7,216477,T. Nedelev,25,Bulgaria,No Club,RM,5'9,154lbs,Left,Medium/ Medium,Normal,NaN,NaN,<NA>,1157750,<NA>,<NA>,68,73,65,66,49,67,53,73,69,68,60,68,78,75,78,58,76,72,54,60,50,67,28,23,62,66,64,59,43,25,22,10,9,8,7,10,1,3,3
8,217225,J. Lalpekhlua,27,India,No Club,RS,5'9,165lbs,Right,Medium/ Medium,Stocky,NaN,NaN,<NA>,422000,<NA>,<NA>,63,64,61,63,57,57,50,61,47,58,46,64,77,74,79,57,80,70,52,60,63,64,47,34,55,46,65,47,28,31,29,13,11,10,10,11,1,3,2
9,236457,D. Giannoulis,22,Greece,No Club,LB,5'10,143lbs,Left,High/ Medium,Lean,NaN,NaN,<NA>,680000,<NA>,<NA>,65,73,62,42,59,55,37,62,44,40,60,58,73,79,64,63,73,65,59,73,47,38,64,65,58,54,47,58,62,62,63,14,5,13,9,12,1,2,2


In [11]:
# View the metadata of the df DataFrame.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18136 entries, 0 to 18135
Data columns (total 56 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        18136 non-null  int64 
 1   name                      18136 non-null  object
 2   age                       18136 non-null  int64 
 3   nationality               18136 non-null  object
 4   club                      18136 non-null  object
 5   position                  18136 non-null  object
 6   height                    18136 non-null  object
 7   weight                    18136 non-null  object
 8   preferred_foot            18136 non-null  object
 9   work_rate                 18136 non-null  object
 10  body_type                 18136 non-null  object
 11  joined                    16643 non-null  object
 12  loaned_from               1264 non-null   object
 13  contract_valid_until      16643 non-null  Int64 
 14  value                 

In [12]:
# Export the updated df DataFrame as players_combined_updated.csv.
df.to_csv('players_combined_updated.csv', index=False)